# Speed Skating Time (500M), 

In honor of the olympics and some good ol' fun, my friends decided to make a bet. The wager is that for any race, I can bet if the first place time is under 41 seconds for the 500M runs. If the skater is under 41 seconds I win, if not I lose. I will model the winning speed skating winning times, and then I can use that to reference the stats of the skaters. This will allow me to get a more informed decision as to weather to take the risk or not.

I will be wrangling the data of short speed skating (500M) times. This data is compiled from across various tournaments.
We need to get this dataset prepped to be modeled.

In [1]:
#Im going to steal my professors datascience libraries off of dropbox
import os
os.chdir(home_path + '/Dropbox/cis399_ds1_f17/week_libraries/datascience_1')
!git pull

NameError: name 'home_path' is not defined

In [2]:
import pandas as pd
import sys

url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQJJH9Kdg4xWsC2BExBi-SgGa2ubQMhirmY-fQtHJBJq0Y4buGvpH8TCdH1WnDIjwZSbGSWusbFiwDk/pub?gid=1995895160&single=true&output=csv'
skate_table = pd.read_csv(url)

pd.__version__  # should see 0.20.3 or higher

#This is the local file of my functions 

home_path =  os.path.expanduser('~')
sys.path.append(home_path + '\Documents\CIS 399\datascience_1')
from week7 import *
%who function

def kmeans_fill(centroids, full_table, focus_table, features, focus_column, k):
    title = "kmeans_" + focus_column
    ohe = pd.get_dummies(full_table[focus_column], prefix = 'k', dummy_na = True)
    full_table = full_table.join(ohe)
    centroids = compute_centroid_labels(centroids, focus_table, focus_column, features, k)
    full_table[title] = full_table.apply(lambda row: centroids[closest_centroid(centroids, row_to_vect(row, features), k)[0]]['mean_label'] if row.time_lap5 == '?' else row.time_lap5, axis = 1)
    full_table.drop(ohe.columns.values, axis = 1, inplace = True) # This is to clean up the table
    return full_table

def compute_centroid_labels(centroids, focus_table, focus_column, features, k):
    for i in range(k):
        centroids[i]['cluster'] = []
    for i in range(len(focus_table)):
        row = focus_table.iloc[i]
        vrow = row_to_vect(row, features)
        (index, dist) = closest_centroid(centroids, vrow, k)
        centroids[index]['cluster'].append(row[focus_column])
    for i in range(k):
        row = centroids[i]['cluster']
        row_sum = 0
        for j in row:
            row_sum += float(j)
        mean = row_sum / len(row)
        centroids[i]['mean_label'] = mean
    return centroids

accuracy	 build_pred	 build_tree_iter	 caser	 closest_centroid	 compute_mean	 compute_prediction	 compute_training	 euclidean_distance	 
f1	 find_best_splitter	 forest_builder	 generate_table	 gig	 gini	 informedness	 initialize_centroids	 k_fold	 
k_means	 phase_1	 phase_2	 predictor_case	 probabilities	 row_to_vect	 seeder	 tree_predictor	 vote_taker	 



In [3]:
pd.set_option('display.max_columns', None)
skate_table.head(10)

,Season,Series,City,Country,Year,Month,Day,Distance,Round,Group,Num_Skater,Name,Nationality,Rank_In_Group,Start_Position,Time,Qualification,rank_lap1,time_lap1,rank_lap2,time_lap2,rank_lap3,time_lap3,rank_lap4,time_lap4,rank_lap5,time_lap5
0,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,1,153,AN Victor',RUS,1,1,41.311,Q,1,6.94,1,8.7,1,8.37,1,8.53,1,8.78
1,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,1,114,SHI Jingnan',CHN,2,3,41.358,Q,2,7.12,2,8.68,2,8.27,2,8.56,2,8.73
2,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,1,129,David Mano LOTH',HUN,3,2,43.338,?,3,7.15,3,9.3,3,8.76,3,8.93,3,9.2
3,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,1,101,Jose Ignacio FAZIO',ARG,?,4,?,DNS,?,?,?,?,?,?,?,?,?,?
4,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,2,155,Vladimir GRIGOREV',RUS,1,3,40.984,Q,2,7.03,2,8.85,2,8.24,2,8.37,1,8.5
5,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,2,165,J R CELSKI',USA,2,1,40.992,Q,1,6.88,1,8.87,1,8.29,1,8.36,2,8.6
6,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,2,103,Nathaniel HENRY',AUS,3,2,42.665,?,3,7.04,3,9.11,3,8.66,3,8.88,3,8.98
7,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,2,186,Yonaton Spencer SUBIN',ISR,4,4,47.282,?,4,7.89,4,9.97,4,9.74,4,9.75,4,9.94
8,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,3,143,LEE Han-Bin',KOR,1,2,41.643,Q,1,7.11,1,9.01,1,8.44,1,8.47,1,8.62
9,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,3,130,BYKANOV Vladislav',ISR,2,4,41.72,Q,4,7.41,3,9.12,2,8.29,2,8.31,2,8.59


## Missing Data Dilemma 

Here we see that there are some occurrences where the racer doesn't have a complete set of times. These occurrences happen if the racer was penalized, did not show, did not finish, and some of the racers that received a 'YC. I do not know what kind of a qualification the 'YC' component entails, so I will treat these values as null.

For those that did not finish, I will fill in their rows with the an average time of the closest data that they would have gotten if they finished. To do this, I will use K-means to fill in the times. 

Now to use this interesting algorithm, I need to prep the data.

In [4]:
#first we have to make sure that we don't have any '?' values in the table when we generate centroids

Non_time_columns = ['Start_Position','City','Country','Qualification','Rank_In_Group']
centroid_table = skate_table.loc[skate_table['Time']!= '?']
for i in range(1,5):
    timelap = 'time_lap' + str(i)
    centroid_table = centroid_table.loc[centroid_table[timelap]!= '?'] 

centroid_table = centroid_table.drop(Non_time_columns,axis = 1)


I chose to drop Qualification and Rank_In_Group from the centroid table because these aspects make it impossible for the
generation of time using k-means as they can hold '?' values, and hold to relationship to the racer's times.

### Generation of data
I will chose to generate the lap times for the fifth lap instead of the total race time itself. If we go for the time value itself I think that this will not be as good of a fit if we choose to go the the fifth lap times and then use that to generate all the total race times. 

I have to perform a check for the rows with values for all but the time for lap 5. Then use the k-means algorithm to fill in the time_lap5. Once that is done I can get more finish times, which will hopefully mean better predictions!

Now, I chose against the generations of more lap times, because I think that generating more data seems a little farfetched, and can cause the data to skew away from the reality.

In [5]:
k = 5
focus_column =  'time_lap5'
features_used = ['time_lap1','time_lap2','time_lap3','time_lap4']

centroids = initialize_centroids(centroid_table, features_used)

skate_temp5 = skate_table
skate_temp5 = skate_temp5.loc[skate_temp5['time_lap1']!= '?']
skate_temp5 = skate_temp5.loc[skate_temp5['time_lap2']!= '?']
skate_temp5 = skate_temp5.loc[skate_temp5['time_lap3']!= '?']
skate_temp5 = skate_temp5.loc[skate_temp5['time_lap4']!= '?']
skate_temp5.loc[skate_temp5['time_lap5'] == '?'].head()

,Season,Series,City,Country,Year,Month,Day,Distance,Round,Group,Num_Skater,Name,Nationality,Rank_In_Group,Start_Position,Time,Qualification,rank_lap1,time_lap1,rank_lap2,time_lap2,rank_lap3,time_lap3,rank_lap4,time_lap4,rank_lap5,time_lap5
31,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,8,133,DOTTI Tommaso',ITA,?,1,?,PEN,3,7.02,4,9.06,4,8.56,3,8.68,?,?
52,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Heats,1,124,WHELBOURNE Jack',GBR,?,4,?,PEN,3,6.93,3,8.71,3,8.14,3,8.26,?,?
72,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Heats,6,143,LEE Han-Bin',KOR,?,3,?,PEN,3,7.07,3,8.76,2,8.24,2,8.49,?,?
85,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Quarterfinals,3,128,LIU Shaolin Sandor',HUN,?,3,?,PEN,2,6.68,2,8.78,2,8.15,2,8.3,?,?
107,2012/2013,World Cup',Calgary,CAN,2012,10,19,500m,Preliminaries,1,92,Ronald CALDECUTT',AUS,?,3,?,PEN,4,7,5,18.98,5,10.55,5,10.02,?,?


In [6]:
skate_table = kmeans_fill(centroids,skate_temp5, centroid_table, features_used, focus_column, k)
skate_table.loc[skate_table['Qualification'] == 'PEN'].head()

,Season,Series,City,Country,Year,Month,Day,Distance,Round,Group,Num_Skater,Name,Nationality,Rank_In_Group,Start_Position,Time,Qualification,rank_lap1,time_lap1,rank_lap2,time_lap2,rank_lap3,time_lap3,rank_lap4,time_lap4,rank_lap5,time_lap5,kmeans_time_lap5
31,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,8,133,DOTTI Tommaso',ITA,?,1,?,PEN,3,7.02,4,9.06,4,8.56,3,8.68,?,?,9.88608
52,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Heats,1,124,WHELBOURNE Jack',GBR,?,4,?,PEN,3,6.93,3,8.71,3,8.14,3,8.26,?,?,9.19076
72,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Heats,6,143,LEE Han-Bin',KOR,?,3,?,PEN,3,7.07,3,8.76,2,8.24,2,8.49,?,?,9.3137
85,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Quarterfinals,3,128,LIU Shaolin Sandor',HUN,?,3,?,PEN,2,6.68,2,8.78,2,8.15,2,8.3,?,?,9.14665
107,2012/2013,World Cup',Calgary,CAN,2012,10,19,500m,Preliminaries,1,92,Ronald CALDECUTT',AUS,?,3,?,PEN,4,7,5,18.98,5,10.55,5,10.02,?,?,9.88608




So we have successfully generated times for those that didn't have a lap 5 time recorded for one reason or another. 

It should be noted that the skate_table returned from the algorithm of k_means removed the rows which had more than 1 missing lap time. I did this intentionally, as those with more than 1 lap time missing should be pruned from our data



In [7]:
for i in range(1,5):
    timelap = 'time_lap' + str(i)
    skate_table = skate_table.loc[skate_table[timelap]!= '?']
    
skate_table['filled_time'] = skate_table.apply(lambda row: (float(row.time_lap1)+float(row.time_lap2)+float(row.time_lap3)+float(row.time_lap4)+float(row.kmeans_time_lap5)) if row.Time == '?' else float(row.Time), axis = 1)
skate_table.head()

,Season,Series,City,Country,Year,Month,Day,Distance,Round,Group,Num_Skater,Name,Nationality,Rank_In_Group,Start_Position,Time,Qualification,rank_lap1,time_lap1,rank_lap2,time_lap2,rank_lap3,time_lap3,rank_lap4,time_lap4,rank_lap5,time_lap5,kmeans_time_lap5,filled_time
0,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,1,153,AN Victor',RUS,1,1,41.311,Q,1,6.94,1,8.7,1,8.37,1,8.53,1,8.78,8.78,41.311
1,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,1,114,SHI Jingnan',CHN,2,3,41.358,Q,2,7.12,2,8.68,2,8.27,2,8.56,2,8.73,8.73,41.358
2,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,1,129,David Mano LOTH',HUN,3,2,43.338,?,3,7.15,3,9.3,3,8.76,3,8.93,3,9.2,9.2,43.338
4,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,2,155,Vladimir GRIGOREV',RUS,1,3,40.984,Q,2,7.03,2,8.85,2,8.24,2,8.37,1,8.5,8.5,40.984
5,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,2,165,J R CELSKI',USA,2,1,40.992,Q,1,6.88,1,8.87,1,8.29,1,8.36,2,8.6,8.6,40.992


Yay! Now we have a table that is ready to be used for random forest predictions. However, this data needs a bit more wrangling.

### Outliers

So make sure this data is going to be good for random forest predictions, we will analyze the outliers to see if there is any truly valuable information to be had within the data. 

In [8]:
sigma3_time =  skate_table['filled_time'].std() * 3 # calculate std * 3
mean_time =   skate_table['filled_time'].mean()# calculate mean
low_time = mean_time - (sigma3_time/3)
high_time = mean_time + sigma3_time
print(mean_time)
print(low_time, high_time)

43.8780404184
(37.63183921526509, 62.616644027722046)


In [9]:
skate_table.loc[skate_table['filled_time'] < low_time].head()

,Season,Series,City,Country,Year,Month,Day,Distance,Round,Group,Num_Skater,Name,Nationality,Rank_In_Group,Start_Position,Time,Qualification,rank_lap1,time_lap1,rank_lap2,time_lap2,rank_lap3,time_lap3,rank_lap4,time_lap4,rank_lap5,time_lap5,kmeans_time_lap5,filled_time


In [10]:
skate_table.loc[skate_table['filled_time'] > high_time].head()

,Season,Series,City,Country,Year,Month,Day,Distance,Round,Group,Num_Skater,Name,Nationality,Rank_In_Group,Start_Position,Time,Qualification,rank_lap1,time_lap1,rank_lap2,time_lap2,rank_lap3,time_lap3,rank_lap4,time_lap4,rank_lap5,time_lap5,kmeans_time_lap5,filled_time
23,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,6,104,BRANKOVIC Edin',BIH,4,3,76.477,?,3,7.51,4,15.67,4,23.64,4,16.9,4,12.76,12.76,76.477000
68,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Heats,5,151,KNEGT Sjinkie',NED,4,3,72.77,?,4,7.2,4,8.84,4,8.2,4,24.74,4,23.79,23.79,72.770000
131,2012/2013,World Cup',Calgary,CAN,2012,10,19,500m,Preliminaries,6,169,Jakub JAWORSKI',POL,4,1,82.201,?,3,7.12,4,15.86,5,36.02,5,13.02,4,10.19,10.19,82.201000
132,2012/2013,World Cup',Calgary,CAN,2012,10,19,500m,Preliminaries,6,179,Jose CAVALLI',SUI,?,3,?,PEN,4,7.27,5,19.27,4,18.07,4,11.99,?,?,9.88608,66.486078
156,2012/2013,World Cup',Calgary,CAN,2012,10,19,500m,Preliminaries,11,139,Edoardo REGGIANI',ITA,5,5,82.165,?,5,7.58,5,9.12,5,8.53,5,36.36,5,20.58,20.58,82.165000


There seems to be quite a bit of the high end outliers. We shall check to make sure there isn't something common amongst these times. Now, I wouldn't expect there to be any of these times to be 3 standard deviations away from the mean on the lower end. So I chose to go with one standard deviation, and there is only one time. I chose to not take this out of the data. 

In [11]:
high_table = skate_table.loc[skate_table['filled_time'] > high_time]
print(high_table.loc[high_table['Time'] == '?'].size/29) #there are 29 columns
print(high_table.size/29) 

39
173


so we only created 39 outliers with our kmeans algorithm. I think that is pretty good. 
There are 173 outliers in the high table... I think I can live with removing 173 outliers from our 5000+ rows. 
About 3% of data lost to outliers.

In [12]:
skate_table = skate_table.loc[skate_table['filled_time'] < high_time]

Now our table has filled times and is removed from outliers. 

In [13]:
temp_table = skate_table.loc[skate_table['filled_time'] < 41]
print(skate_table.size/29)
temp_table.size/29

4801


333

,Season,Series,City,Country,Year,Month,Day,Distance,Round,Group,Num_Skater,Name,Nationality,Rank_In_Group,Start_Position,Time,Qualification,rank_lap1,time_lap1,rank_lap2,time_lap2,rank_lap3,time_lap3,rank_lap4,time_lap4,rank_lap5,time_lap5,kmeans_time_lap5,filled_time,winning
0,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,1,153,AN Victor',RUS,1,1,41.311,Q,1,6.94,1,8.7,1,8.37,1,8.53,1,8.78,8.78,41.311,0
1,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,1,114,SHI Jingnan',CHN,2,3,41.358,Q,2,7.12,2,8.68,2,8.27,2,8.56,2,8.73,8.73,41.358,0
2,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,1,129,David Mano LOTH',HUN,3,2,43.338,?,3,7.15,3,9.3,3,8.76,3,8.93,3,9.2,9.2,43.338,0
4,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,2,155,Vladimir GRIGOREV',RUS,1,3,40.984,Q,2,7.03,2,8.85,2,8.24,2,8.37,1,8.5,8.5,40.984,1
5,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,2,165,J R CELSKI',USA,2,1,40.992,Q,1,6.88,1,8.87,1,8.29,1,8.36,2,8.6,8.6,40.992,1


In [15]:
import os

home_path =  os.path.expanduser('~')

file_path = '\Documents\CIS 399\ '

file_name = 'final_project_wrangled.csv'

skate_table.to_csv(home_path + file_path + file_name, index=False)